In [1]:
import sys
from torch.nn.utils.rnn import pad_sequence
# sys.path.append("./Incontext-learning") # this part works for goole colab

In [2]:
import utility
import metric
import importlib
importlib.reload(utility)
importlib.reload(metric)
from utility import (data_selection, evaluate_zeroshot,evaluate_finetuning,evaluate_demonstrations as  evaluate_fewshots, finetune_model_eval)
from metric import Rec2FTP, SimAOU


In [3]:
import transformers
import torch

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# check point

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load SST-2 dataset
# dataset = load_dataset("glue", "sst2")

# load SST-5 dataset
dataset = load_dataset("SetFit/sst5")

# Load Qwen3 tokenizer and model
model_name = "Qwen/Qwen3-0.6B"
model_path = "/home/students/wli/UniHeidelberg/semster2/final_projects/models/Qwen3-0.6B-Base"
# model_path = model_name
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map = "auto")

tokenizer.padding_side = "left"
# Make sure tokenizer has pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# if getattr(model.config.pad_token_id, None) is None:
    # pass

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})

In [7]:
! nvidia-smi

Thu Sep 18 15:47:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off |   00000000:3B:00.0 Off |                  N/A |
| 25%   38C    P5             15W /  250W |    1295MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
train_dataset = dataset["train"] # the orginal paper use this
test_dataset = dataset["validation"]

def add_idx(example, idx):
    example["idx"] = idx
    return example

test_dataset = test_dataset.map(add_idx, with_indices=True)
# Format examples as causal LM inputs
def preprocess_function(examples):
    label_map = {0: "negative", 1: "positive"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["sentence"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

def preprocess_function_sst5(examples):
    label_map = {0: "terrible", 1: "bad", 2: "neutral", 3: "good", 4: "very great"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["text"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


# Tokenize the dataset
# tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
train_tokenized_datasets = train_dataset.map(preprocess_function_sst5, batched = True, remove_columns = train_dataset.column_names)
test_tokenized_datasets = test_dataset.map(preprocess_function_sst5, batched=True, remove_columns=test_dataset.column_names)
# Data collator (handles padding dynamically in batch)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [9]:
# test the behavior of LLM before fine tuing
prompt = "Sentence: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films. Label:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# You can control max_length or early stopping
output_ids = model.generate(
    **inputs,
    max_new_tokens=1,    # adjust based on expected label length
    do_sample=False,       # greedy decoding
    pad_token_id=tokenizer.pad_token_id
)

# Decode generated tokens
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

Sentence: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films. Label: "


In [10]:
test_dataset

Dataset({
    features: ['text', 'label', 'label_text', 'idx'],
    num_rows: 1101
})

In [11]:
a, b = SimAOU(model,tokenizer,train_dataset, test_dataset, best_seed = 0, num_data_points = 4, data_type = "sst5")

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.11 GiB. GPU 0 has a total capacity of 10.90 GiB of which 403.06 MiB is free. Including non-PyTorch memory, this process has 10.51 GiB memory in use. Of the allocated memory 10.33 GiB is allocated by PyTorch, and 16.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
finetune_model_eval(model, tokenizer,train_dataset,test_dataset, best_seed = 50, data_type="sst5", num_data_points = 32)

Accuracy is 0.0
Accuracy is 0.4659400544959128


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss


training finished!
Accuracy is 0.3278837420526794
 The Rec2FT is 1.3749999095394796


1.3749999095394796

In [ ]:
# icl outcome 
outcome, candidate, demonstrations = data_selection(model, tokenizer, train_dataset,test_dataset, num_data_points=32, seed_max = 100,batch_size=32,data_type="sst5")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:20<?, ?it/s]


In [ ]:
_, cnt_zsl = evaluate_zeroshot(model, tokenizer, test_dataset, data_type = "sst2")
_, cnt_icl = evaluate_fewshots(model,tokenizer, demonstrations, test_dataset, "sst2", batch_size=64)

In [ ]:
cnt_zsl

In [ ]:
# fine tune the model

def custom_collator(features):
    input_ids = [torch.tensor(f["input_ids"], dtype=torch.long) for f in features]
    attention_mask = [torch.tensor(f["attention_mask"], dtype=torch.long) for f in features]
    labels = [torch.tensor(f["labels"], dtype=torch.long) for f in features]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

data_collator = custom_collator

# Training arguments
training_args = TrainingArguments(
    output_dir="./qwen3_sst2_lm",
    eval_strategy="steps",
    eval_steps=4,
    # save_steps=500,
    logging_steps=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    # gradient_accumulation_steps=,
    num_train_epochs=1,
    learning_rate=5e-5,
    # weight_decay=0.01,
    # save_total_limit=1,
    save_strategy = "no",
    # fp16=True,
    # push_to_hub=False,
    report_to="none"
)


# ** To do fine tuning here only need to fine tune the Key and Value
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=demonstrations.map(preprocess_function, batched=True, remove_columns=demonstrations.column_names),
    eval_dataset=test_tokenized_datasets,
    # tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training
trainer.train()


In [ ]:
_, cnt_ft = evaluate_finetuning(model, tokenizer, test_dataset, data_type="sst2")
metric =len(((set(cnt_ft) - set(cnt_zsl)) & (set(cnt_icl)-set(cnt_zsl)))) \
   /(len((set(cnt_ft) - set(cnt_icl))) + 1e-5)

print(metric)


In [ ]:
metric

In [ ]:
prompt = "Sentence: macdowell , whose wifty southern charm has anchored lighter affairs ... brings an absolutely riveting conviction to her role .  Label:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# You can control max_length or early stopping
output_ids = model.generate(
    **inputs,
    max_new_tokens=1,    # adjust based on expected label length
    do_sample=False,       # greedy decoding
    pad_token_id=tokenizer.pad_token_id
)

# Decode generated tokens
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)


Chooes 32 demonstration examples that can achieve the best validation porformance.

In order to make questions much easier, I split the validation dataset into the train and test set. And to find the 32 examples that can achiveve the best porformance on the test data.